# Defining caffenet

In [ ]:
from ext.network import Network
import tensorflow as tf

class CaffeNet(Network):
    def setup(self):
        (self.feed('input') # (1,227,227,3)
             .conv(11, 11, 96, 4, 4, padding='VALID', name='conv1') # (1, 55, 55, 96)
             .max_pool(3, 3, 2, 2, padding='VALID', name='pool1') # (1, 27, 27, 96)
             .lrn(2, 2e-05, 0.75, name='norm1') 
             .conv(5, 5, 256, 1, 1, group=2, name='conv2') #(1, 27, 27, 256)
             .max_pool(3, 3, 2, 2, padding='VALID', name='pool2') # (1, 13, 13, 256)
             .lrn(2, 2e-05, 0.75, name='norm2')
             .conv(3, 3, 384, 1, 1, name='conv3') # (1, 13, 13, 384)
             .conv(3, 3, 384, 1, 1, group=2, name='conv4')
             .conv(3, 3, 256, 1, 1, group=2, name='conv5') # (1, 13, 13, 256)
             .max_pool(3, 3, 2, 2, padding='VALID', name='pool5') # (1, 6, 6, 256)
             .fc(4096, name='fc6') 
             .fc(4096, name='fc7')
             .fc(1000, relu=False, name='fc8')
             .softmax(name='prob'))

tf.reset_default_graph()
images = tf.placeholder(tf.float32, [1, 227, 227, 3])
labels = tf.placeholder(tf.float32, [1, 1000])
net = CaffeNet({'input':images})

In [ ]:
net.inv_setup()

In [ ]:
net.inv_layers

In [ ]:
invout = net.get_inv_output()
invout

# loading
- load pre-trained weights
- load input tensor

In [ ]:
import tensorflow as tf
import numpy as np
import pickle

f=open('../shared/data_blob.npy','rb')
data_blob = pickle.load(f)
data_blob = data_blob.transpose((0,2,3,1))

sess=tf.InteractiveSession()
net.load('../caffe-tensorflow/caffenet.npy', sess)

# visualization checking

In [ ]:
print net.inv_layers[layer_name]
print net.layers[layer_name]

In [ ]:
from ext.util import *
%pylab inline
layer_name = 'fc8'

layer_output = sess.run(net.layers[layer_name],feed_dict={images:data_blob})
channelId=290

to_vis=np.zeros_like(layer_output)
to_vis[0,...,channelId]=layer_output[0,...,channelId]

figure(figsize=(10,10))

generated = invout.eval(feed_dict={net.inv_layers[layer_name]:to_vis,images:data_blob})
gen_img=ensure_uint255(norm01c(generated[0,:],0))
imshow(gen_img[:,:,::-1],interpolation='nearest')